In [1]:
import sys
sys.path.insert(0, r'c:\users\carte\appdata\local\programs\python\python310\lib\site-packages')

In [2]:
import nltk
from wikidata.client import Client
from wikidata.entity import Entity
from collections import deque
import requests, json
import pandas as pd
import numpy as np
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

MAX_LEVEL = 2

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\carte\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\carte\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
def get_nouns(sentence):
    tokens = nltk.word_tokenize(sentence)
    tags = nltk.pos_tag(tokens)
    retVal = []
    i = 0
    while i < len(tags):
        key, tag = tags[i]
        if 'NN' in tag:
            #current_noun = key
            #j = i + 1
            #while j < len(tags):
            #    key_j, tag_j = tags[j]
            #    if 'NN' in tag_j:
            #        current_noun = ' '.join([current_noun, key_j])
            #        j += 1
            #    else:
            #        i = j - 1
            #        break
            #retVal.append(current_noun)
            #if j == len(tags):
            #    return retVal
            retVal.append(key)
        i += 1
    return retVal

In [5]:
def get_wikidata_id(item):
    try:
        response = requests.get(f'https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&titles={item}&format=json')
        wikidata_id = list(json.loads(response.text)['query']['pages'].values())[0]['pageprops']['wikibase_item']
        return wikidata_id
    except:
        return None


In [9]:
embeddings_np = np.load(r"D:\SDS\KGA_Bert\data\KG_data\wiki_trans_v1_vec.npy")

MemoryError: Unable to allocate 58.4 GiB for an array with shape (15682637000,) and data type float32

In [10]:
def get_training_embeddings(labels_file):

    def __clean_str__(l : str):
        if '@en' in l:
            l = l.replace('@en', '')
        l = l.replace('"', '')
        return l.lower()
    
    retVal = {}
    with open(labels_file) as labels:
        for line in labels:
            curr_label, line_num = line.split('\t')
            line_num = int(line_num)
            curr_label = __clean_str__(curr_label)
            retVal[curr_label] = embeddings_np[line_num]
    return retVal

In [15]:
label_to_embedding = get_training_embeddings(r'C:\Users\carte\OneDrive\Desktop\Datascience\SDS\KGA_Bert\data\KG_data\english_labels.tsv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\carte\\OneDrive\\Desktop\\Datascience\\SDS\\KGA_Bert\\data\\KG_data\\english_labels.tsv'

In [16]:
def contains(noun):
    return noun.title() in label_to_embedding or noun.lower() in label_to_embedding or noun.upper() in label_to_embedding

def get_val(ent):
    val = label_to_embedding.get(str(ent.label))
    if val is None:
        val = label_to_embedding.get(str(ent.label).lower())
    if val is None:
        val = label_to_embedding.get(str(ent.label).upper())
    return val

In [17]:
def bfs(noun):
    client = Client()
    id = get_wikidata_id(noun)
    e = client.get(id)
    seen = set()
    q = deque([(e, 0)])
    averages = {}
    num_elems = [0] * 3
    while q:
        ent, level = q.popleft()
        if ent in seen or level > MAX_LEVEL:
            continue
        seen.add(ent)
        try:
            if isinstance(ent, Entity) and contains(str(ent.label)):
                if level not in averages:
                    averages[level] = get_val(ent)
                else:
                    averages[level] += get_val(ent)
                num_elems[level] += 1

                e = list(client.get(ent.id).values())[:min(len(e), 20)]
                for ent in e:
                    q.append((ent, level + 1))
        except:
            continue
    return {level : np.vectorize(lambda vals : round(vals, 4))(averages[level] / num_elems[level]) for level in range(3)}

In [20]:
data_train = pd.read_csv(r'D:\SDS\KGA_Bert\data\glue_data\SST-2\train.tsv', sep='\t', header=0)
data_test = pd.read_csv(r'D:\SDS\KGA_Bert\data\glue_data\SST-2\dev.tsv', sep='\t', header=0)


nouns = set()
for sentence in data_train['sentence']:
    for noun in get_nouns(sentence):
        nouns.add(noun)

for sentence in data_test['sentence']:
    for noun in get_nouns(sentence):
        nouns.add(noun)

In [ ]:
nouns_with_embeddings = {word for word in nouns if word in label_to_embedding}
nouns_with_embeddings, len(nouns_with_embeddings)

In [ ]:
embeddings_dict = {}
for noun in list(nouns_with_embeddings)[:3]:
    entities = bfs(noun)
    embeddings_dict[noun.lower()] = [entities[0], entities[1], entities[2]]

pd.DataFrame.from_dict(embeddings_dict, orient='index', columns=['1', '2', '3'])

In [21]:
from tqdm import tqdm
for noun in tqdm(nouns):
    client = Client()
    id = get_wikidata_id(noun)
    e = client.get(id)

  2%|▏         | 186/10226 [00:37<32:50,  5.10it/s] 